<a href="https://colab.research.google.com/github/jeraldkim/DataScienceProjects/blob/main/Scraping_Anime_from_myanimelist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re
import time
import numpy as np

In [ ]:
anime_list = []
num_anime = 200 #ex: 5 means 5*50 we want top 250 anime 0 would mean only top 50 anime so not run loop 200 = top 10000 anime
n = 0

while n < num_anime:
    if n == 0:
      curr_link = "https://myanimelist.net/topanime.php"
    elif n > 0:
      curr_link = "https://myanimelist.net/topanime.php?limit=" + str(n * 50)
    n += 1

    response = requests.get(curr_link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")
    table = soup.find("table", class_="top-ranking-table")
    time.sleep(3) #necessary condition to prevent site from blocking your requests


    if table == None:
      print(curr_link)
      print(soup)
      bug1 = response
      bug2 = html_content
      bug3 = soup
    
    for row in table.find_all("tr")[1:51]:
      anime = {}
      anime["Rank"] = int(row.find("td", class_="rank").text)
      anime["Title"] = row.find("div", class_="di-ib clearfix").find("a").text
      anime["Link"] = row.find("a").get("href")
      anime["Score"] = float(row.find("td", class_="score").text)
      anime_list.append(anime)

anime_df = pd.DataFrame(anime_list)

In [19]:
anime_df = pd.DataFrame(anime_list)
anime_df.head(10)

,Rank,Title,Link,Score
0,1,Fullmetal Alchemist: Brotherhood,https://myanimelist.net/anime/5114/Fullmetal_A...,9.11
1,2,Bleach: Sennen Kessen-hen,https://myanimelist.net/anime/41467/Bleach__Se...,9.10
2,3,Steins;Gate,https://myanimelist.net/anime/9253/Steins_Gate,9.08
3,4,Gintama°,https://myanimelist.net/anime/28977/Gintama°,9.07
4,5,Kaguya-sama wa Kokurasetai: Ultra Romantic,https://myanimelist.net/anime/43608/Kaguya-sam...,9.07
5,6,Shingeki no Kyojin Season 3 Part 2,https://myanimelist.net/anime/38524/Shingeki_n...,9.06
6,7,Gintama: The Final,https://myanimelist.net/anime/39486/Gintama__T...,9.05
7,8,Gintama',https://myanimelist.net/anime/9969/Gintama,9.04
8,9,Hunter x Hunter (2011),https://myanimelist.net/anime/11061/Hunter_x_H...,9.04
9,10,Gintama': Enchousen,https://myanimelist.net/anime/15417/Gintama__E...,9.03


In [10]:
from google.colab import drive

In [12]:
drive.mount('/content/drive')
path = '/content/drive/My Drive/10000anime021723part2.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  anime_df.to_csv(f)

#method to save df to csv file on google drive

Mounted at /content/drive


In [22]:
new_anime_list = []

real = anime_df["Link"] #for generating entire data

tests = anime_df["Link"][:10] #use this one to test for implementation

for link in tests: #change here for testing

    response = requests.get(link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")
    time.sleep(3)

    anime = {}
    anime_type = soup.find("span",class_ ="dark_text",string="Type:")
    if anime_type == None:
      anime["Type"] = None
    else:
      anime["Type"] = anime_type.find_parent().text.replace('Type:','').strip() #Type of Show
    num_episodes = soup.find("span",class_ ="dark_text",string="Episodes:")
    if num_episodes == None:
      anime["# Episodes"] = None
    else:
      anime["# Episodes"] = num_episodes.next_sibling.strip() #Number of Episodes
    
    source = soup.find("span",class_ ="dark_text",string="Source:")
    if source == None:
      anime["Source"] = None
    else:
      anime["Source"] = source.next_sibling.strip() #Source

    premiered = soup.find("span",class_ ="dark_text",string="Premiered:")
    if premiered == None:
      anime["Premiered"] = None
    else:
      anime["Premiered"] = premiered.find_parent().find("a").text #Season of Anime and Year

    
    aired_date = soup.find("span",class_ ="dark_text",string="Aired:")
    if aired_date == None:
      anime["Aired Date"] = None
    else:
      anime["Aired Date"] = aired_date.next_sibling.strip() #Aired Date
    
    studio = soup.find("span",class_ ="dark_text",string="Studios:")
    if studio == None:
      anime["Studio"] = None
    else: 
      anime["Studio"] = re.sub('\nStudios:\n','',studio.find_parent().text) #Studio RECENTLy made a regex change for animes with more than 1 studio (rare though)

    genre = soup.find("span",class_ ="dark_text",string=re.compile(r'Genres:|Genre:'))
    if genre == None:
      anime["Genre"] = None
    else:
      anime["Genre"] = genre.find_parent().find("a").text #Genre


    demo = soup.find("span",class_ ="dark_text",string="Demographic:")
    if demo == None:
      anime["Demographic"] = None
    else:
      anime["Demographic"] = demo.find_parent().find("a").text #Demographic
    
    duration = soup.find("span",class_ ="dark_text",string="Duration:")
    if duration == None:
      anime["Duration"] = None
    else:
      anime["Duration"] = duration.next_sibling.strip() #Duration

    age_rating = soup.find("span",class_ ="dark_text",string="Rating:")
    if age_rating == None:
      anime["Age Rating"] = None
    else:
      anime["Age Rating"] = age_rating.next_sibling.strip() #Age Rating

    #New feature additions started from this line
    num_raters = soup.find("span",class_ ="dark_text",string="Score:")
    if num_raters == None:
      anime["num_raters"] = None
    else:
      anime["num_raters"] = num_raters.find_parent().find(itemprop="ratingCount").text #Number of people who rated the scores

    popularity = soup.find("span",class_ ="dark_text",string="Popularity:")
    if popularity == None:
      anime["Popularity"] = None
    else:
      anime["Popularity"] = popularity.next_sibling.strip() #Popularity Rank determined by number of "members"

    members = soup.find("span",class_ ="dark_text",string="Members:")
    if members == None:
      anime["Members"] = None
    else:
      anime["Members"] = members.next_sibling.strip() # Number of members = num of  people who added the series to the list


    favorites = soup.find("span",class_ ="dark_text",string="Favorites:")
    if favorites == None:
      anime["Favorites"] = None
    else:
      anime["Favorites"] = favorites.next_sibling.strip() #Favorites

    #new addition of addign voice actor, character, role

    Character = []
    Role = []
    Voice_Actors = []

    characters = soup.find("div",class_="detail-characters-list clearfix")
    for child in characters.children:

      for chars in child.find_all("h3"):
        Character += [chars.text]

      for chars in child.find_all("div",class_ = "spaceit_pad"):
        Role += [chars.text.strip()]

      for chars in child.find_all("td",class_ = "va-t ar pl4 pr4"):
        Voice_Actors += [re.sub('\\n.*','',chars.text.strip())]

    anime["Characters"] = Character
    anime["Role"] = Role
    anime["Voice Actors"] =  Voice_Actors   
    # list_of_tuples = list(zip(Character, Role, Voice_Actor))

    # anime["Characters/Role/Voice_Actors"] = pd.DataFrame(list_of_tuples,columns=['Character', 'Role','Voice_Actors'])
    
    #Final Step
    new_anime_list.append(anime)    

new_anime_df = pd.DataFrame(new_anime_list)

# drive.mount('/content/drive')
# path = '/content/drive/My Drive/10000anime_more_stats_021723.csv'

# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   new_anime_df.to_csv(f)

In [16]:
drive.mount('/content/drive')
path = '/content/drive/My Drive/3980anime_more_stats_021723.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  test_df.to_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
test_df = pd.DataFrame(new_anime_list)
test_df

,Type,# Episodes,Source,Premiered,Aired Date,Studio,Genre,Demographic,Duration,Age Rating,num_raters,Popularity,Members,Favorites,Characters,Role,Voice Actors
0,TV,64,Manga,Spring 2009,"Apr 5, 2009 to Jul 4, 2010",Bones,Action,Shounen,24 min. per ep.,R - 17+ (violence & profanity),1964664,#3,"3,090,036","213,378","[Elric, Edward, Elric, Alphonse, Mustang, Roy,...","[Main, Main, Supporting, Supporting, Supportin...","[Park, Romi, Kugimiya, Rie, Miki, Shinichiro, ..."
1,TV,13,Manga,Fall 2022,"Oct 11, 2022 to Dec 27, 2022",Pierrot,Action,Shounen,24 min. per ep.,R - 17+ (violence & profanity),171327,#526,"389,256","15,769","[Kurosaki, Ichigo, Kuchiki, Rukia, Inoue, Orih...","[Main, Main, Main, Main, Main, Main, Supportin...","[Morita, Masakazu, Orikasa, Fumiko, Matsuoka, ..."
2,TV,24,Visual novel,Spring 2011,"Apr 6, 2011 to Sep 14, 2011",White Fox,Drama,None,24 min. per ep.,PG-13 - Teens 13 or older,1305085,#13,"2,380,196","179,577","[Okabe, Rintarou, Makise, Kurisu, Shiina, Mayu...","[Main, Main, Main, Main, Supporting, Supportin...","[Miyano, Mamoru, Imai, Asami, Hanazawa, Kana, ..."
3,TV,51,Manga,Spring 2015,"Apr 8, 2015 to Mar 30, 2016",Bandai Namco Pictures,Action,Shounen,24 min. per ep.,PG-13 - Teens 13 or older,231470,#336,"578,117","15,541","[Sakata, Gintoki, Kagura, Shimura, Shinpachi, ...","[Main, Main, Main, Supporting, Supporting, Sup...","[Sugita, Tomokazu, Kugimiya, Rie, Sakaguchi, D..."
4,TV,13,Manga,Spring 2022,"Apr 9, 2022 to Jun 25, 2022",A-1 Pictures,Comedy,Seinen,23 min. per ep.,PG-13 - Teens 13 or older,405699,#226,"757,057","27,211","[Shinomiya, Kaguya, Ishigami, Yuu, Fujiwara, C...","[Main, Main, Main, Main, Main, Supporting, Sup...","[Koga, Aoi, Suzuki, Ryouta, Kohara, Konomi, Fu..."
5,TV,10,Manga,Spring 2019,"Apr 29, 2019 to Jul 1, 2019",Wit Studio,Action,Shounen,23 min. per ep.,R - 17+ (violence & profanity),1418097,#27,"2,024,469","54,027","[Levi, Yeager, Eren, Ackerman, Mikasa, Arlert,...","[Main, Main, Main, Main, Supporting, Supportin...","[Kamiya, Hiroshi, Kaji, Yuuki, Ishikawa, Yui, ..."
6,Movie,1,Manga,None,"Jan 8, 2021",Bandai Namco Pictures,Action,Shounen,1 hr. 44 min.,PG-13 - Teens 13 or older,59465,#1591,"124,354","3,956","[Sakata, Gintoki, Kagura, Shimura, Shinpachi, ...","[Main, Main, Main, Supporting, Supporting, Sup...","[Sugita, Tomokazu, Kugimiya, Rie, Sakaguchi, D..."
7,TV,51,Manga,Spring 2011,"Apr 4, 2011 to Mar 26, 2012",Sunrise,Action,Shounen,24 min. per ep.,PG-13 - Teens 13 or older,220638,#384,"509,809","7,549","[Sakata, Gintoki, Kagura, Shimura, Shinpachi, ...","[Main, Main, Main, Supporting, Supporting, Sup...","[Sugita, Tomokazu, Kugimiya, Rie, Sakaguchi, D..."
8,TV,148,Manga,Fall 2011,"Oct 2, 2011 to Sep 24, 2014",Madhouse,Action,Shounen,23 min. per ep.,PG-13 - Teens 13 or older,1601626,#10,"2,577,282","195,449","[Zoldyck, Killua, Kurapika, Freecss, Gon, Pala...","[Main, Main, Main, Main, Supporting, Supportin...","[Ise, Mariya, Sawashiro, Miyuki, Han, Megumi, ..."
9,TV,13,Manga,Fall 2012,"Oct 4, 2012 to Mar 28, 2013",Sunrise,Action,Shounen,24 min. per ep.,PG-13 - Teens 13 or older,154092,#712,"301,217","2,927","[Sakata, Gintoki, Kagura, Shimura, Shinpachi, ...","[Main, Main, Main, Supporting, Supporting, Sup...","[Sugita, Tomokazu, Kugimiya, Rie, Sakaguchi, D..."


In [112]:
# prime_anime_df = anime_df.iloc[:932].join(test_df)

In [ ]:
# np.unique(prime_anime_df["Type"])

In [ ]:
# np.unique(prime_anime_df["Source"])

In [ ]:
# prime_anime_df[prime_anime_df["Source"]=="Web manga"]

In [ ]:
anime_df.iloc[3980:]